## Mask Detection with ResNet

In [1]:
# Import libraries:

import shutil
import os
import numpy as np
import cv2
from keras.applications.resnet import ResNet50, preprocess_input
from keras.models import Model
from keras.layers import Dense,Dropout,AveragePooling2D,Flatten
from keras.optimizers import Adam
from tqdm.notebook import tqdm_notebook
import datetime
import tensorflow as tf
from focal_loss import BinaryFocalLoss
from tqdm import tqdm_notebook as tqdm
from PIL import Image
from sklearn.utils import shuffle

In [2]:
# Load the TensorBoard notebook extension:
%load_ext tensorboard

In [3]:
# Make use of TensorBoard:

try:
    shutil.rmtree('logs')
except:
    pass
try:
    shutil.rmtree('logsx')
except:
    pass

# Try with TB:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


In [5]:
# Import Image Dataset from local:

def read_images(path):
    files = os.listdir(path)
    files = [file for file in files if file[-4:] == ".jpg"]
        
    images = []
    for file in tqdm(files):
        try:
            image = Image.open(os.path.join(path, file))

            # https://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.resize
            image = image.resize((224, 224), Image.LANCZOS)

            # https://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.convert
            image = image.convert("RGB")

            image = np.asarray(image)

            images.append(image)
        except OSError:
            pass
    
    return images
    
with_mask = read_images(os.path.join("./dataset/cropped/", "with"))
without_mask = read_images(os.path.join("./dataset/cropped/", "without"))

C:\Users\Slavka\AppData\Local\Temp\ipykernel_22192\3113043258.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for file in tqdm(files):


  0%|          | 0/2332 [00:00<?, ?it/s]

C:\Users\Slavka\AppData\Local\Temp\ipykernel_22192\3113043258.py:13: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  image = image.resize((224, 224), Image.LANCZOS)


  0%|          | 0/1330 [00:00<?, ?it/s]

In [6]:
# Prepare Image Data in Data X and Target Values y:

with_mask = np.asarray(with_mask)
without_mask = np.asarray(without_mask)

X = np.concatenate([with_mask, without_mask])

y_with_mask = np.zeros(len(with_mask))
y_without_mask = np.ones(len(without_mask))

y = np.concatenate([y_with_mask, y_without_mask])

del with_mask, without_mask, y_with_mask, y_without_mask

## Transfer Learning with ResNet

In [7]:
# Use ResNet Model for Transfer-Learning with non-trainable layers for faster processing:

resnet_model  = ResNet50(input_shape = (224,224,3), weights='imagenet', include_top=False)
for layer in resnet_model.layers:
  layer.trainaibale = False        

classifier_network = resnet_model.output
classifier_network = AveragePooling2D(pool_size=(7, 7),name = "Average_Pool_Final")(classifier_network)
classifier_network = Flatten(name = "Flatten_Final")(classifier_network)
classifier_network = Dense(128, activation="relu",name = "Dense_Final")(classifier_network)
classifier_network = Dropout(0.5,name = "Dropout_Final")(classifier_network)
classifier_network = Dense(1, activation="sigmoid", name = "Sigmoid_Classifier" )(classifier_network)

mask_classifier = Model(inputs=resnet_model.input, outputs=classifier_network)

In [8]:
# Set the Hyper-Parameters

alpha = 0.00001
n_epochs = 25
mini_batch_size = 32
     

In [9]:
# Compile the Model

opt = Adam(learning_rate=alpha,decay = alpha/n_epochs)

#mask_classifier.compile(optimizer=Adam(learning_rate=0.0001), loss="binary_crossentropy", metrics=["acc"])
mask_classifier.compile(optimizer=opt,loss=BinaryFocalLoss(gamma=2),metrics = ['accuracy'])

#mask_classifier.summary() 

In [10]:
# Use Preprocessing of keras ResNet Library and shuffle Dataset afterwards for better Predictions:

X = preprocess_input(X)
X, y = shuffle(X, y)

In [11]:
# Train the Model

mask_classifier.fit(X, y, epochs=n_epochs, batch_size=mini_batch_size, validation_split=0.2, shuffle=True, callbacks=[tensorboard_callback])

Epoch 1/25
92/92 [==============================] - 24s 171ms/step - loss: 0.1976 - accuracy: 0.7316 - val_loss: 0.0463 - val_accuracy: 0.9222
Epoch 2/25
92/92 [==============================] - 15s 157ms/step - loss: 0.0707 - accuracy: 0.8867 - val_loss: 0.0265 - val_accuracy: 0.9604
Epoch 3/25
92/92 [==============================] - 15s 156ms/step - loss: 0.0439 - accuracy: 0.9324 - val_loss: 0.0206 - val_accuracy: 0.9714
Epoch 4/25
92/92 [==============================] - 15s 158ms/step - loss: 0.0276 - accuracy: 0.9597 - val_loss: 0.0178 - val_accuracy: 0.9741
Epoch 5/25
92/92 [==============================] - 15s 157ms/step - loss: 0.0191 - accuracy: 0.9730 - val_loss: 0.0164 - val_accuracy: 0.9795
Epoch 6/25
92/92 [==============================] - 15s 157ms/step - loss: 0.0142 - accuracy: 0.9819 - val_loss: 0.0151 - val_accuracy: 0.9823
Epoch 7/25
92/92 [==============================] - 15s 157ms/step - loss: 0.0137 - accuracy: 0.9822 - val_loss: 0.0140 - val_accuracy: 0.9864

In [12]:
# And finally save the Model:

mask_classifier.save("./model/resnet.h5")